
# Conversión de Cajas de Porcentaje a Metros y Cálculo de Estadísticas

Este cuaderno realiza los siguientes pasos:

1. Carga un archivo CSV con columnas `xmin`, `ymin`, `xmax`, `ymax` en valores porcentuales (0–100).
2. Convierte estas coordenadas a metros, utilizando las dimensiones de la imagen y la relación de conversión (1 píxel = 3.197 cm).
3. Calcula el ancho (`width_m`), alto (`height_m`) y área (`area_m2`) en metros de cada caja.
4. Guarda un nuevo CSV con las columnas originales más las columnas convertidas a metros.
5. Calcula estadísticas de las cajas:
   - Caja(s) con ancho y alto mínimos simultáneamente.
   - Caja(s) con ancho y alto máximos simultáneamente.
   - Caja(s) con área mínima.
   - Caja(s) con área máxima.


In [2]:

def percent_x_to_m(x_percent, img_width_pixels, px_to_cm):
    """
    Convierte un porcentaje en el eje X (0-100) a metros.
    1. Calcula píxeles: (x_percent / 100) * ancho_imagen_píxeles
    2. Convierte píxeles a cm: píxeles * px_to_cm
    3. Convierte cm a metros: dividir por 100
    """
    x_pixels = (x_percent / 100.0) * img_width_pixels
    x_cm = x_pixels * px_to_cm
    return x_cm / 100.0  # Convertir de cm a metros

def percent_y_to_m(y_percent, img_height_pixels, px_to_cm):
    """
    Convierte un porcentaje en el eje Y (0-100) a metros.
    1. Calcula píxeles: (y_percent / 100) * alto_imagen_píxeles
    2. Convierte píxeles a cm: píxeles * px_to_cm
    3. Convierte cm a metros: dividir por 100
    """
    y_pixels = (y_percent / 100.0) * img_height_pixels
    y_cm = y_pixels * px_to_cm
    return y_cm / 100.0  # Convertir de cm a metros


In [3]:

import pandas as pd
import os
import rasterio

# Verificar existencia del archivo CSV
input_path = "../data/database/test/tif_1x_res/"  # Ajustar si el nombre o ruta cambian
if not os.path.isfile(input_path):
    raise FileNotFoundError(f"No se encontró el archivo CSV en la ruta: {input_path}")
    
# Cargar el CSV original
df = pd.read_csv(input_path)

# Folder donde se encuentran las imágenes
image_folder = "../data/labels/labels_tifname_porcentual.csv"  

# Factor de conversión de píxel a centímetros
px_to_cm = 3.197000000000000536  # Centímetros por píxel

# Crear listas para almacenar las dimensiones de la imagen
xmin_m, xmax_m, ymin_m, ymax_m = [], [], [], []
width_m, height_m, area_m2 = [], [], []
image_names, labels = [], []

# Iterar sobre cada imagen dentro del folder
for filename in os.listdir(image_folder):
    if filename.endswith(".tif"):
        # Obtener el nombre de la imagen sin extensión
        image_name = filename
        
        # Filtrar el DataFrame para obtener las filas que contengan el nombre de la imagen
        img_df = df[df["image_path"] == image_name]
        
        # Cargar la imagen para obtener sus dimensiones
        img_path = os.path.join(image_folder, filename)
        # abrir la imagen tipo raster
        img = rasterio.open(img_path)
        img_width_pixels, img_height_pixels = img.width, img.height
            
        # Convertir coordenadas de porcentaje a metros
        for index, row in img_df.iterrows():
            xmin_m.append(percent_x_to_m(row["xmin"], img_width_pixels, px_to_cm))
            xmax_m.append(percent_x_to_m(row["xmax"], img_width_pixels, px_to_cm))
            ymin_m.append(percent_y_to_m(row["ymin"], img_height_pixels, px_to_cm))
            ymax_m.append(percent_y_to_m(row["ymax"], img_height_pixels, px_to_cm))
            width_m.append(xmax_m[-1] - xmin_m[-1])
            height_m.append(ymax_m[-1] - ymin_m[-1])
            area_m2.append(width_m[-1] * height_m[-1])
            image_names.append(image_name)
            labels.append(row["label"])

# Crear un nuevo DataFrame con las coordenadas convertidas y agregar la etiqueta
df_converted = pd.DataFrame({
    "img_name": image_names,
    "label": labels,
    "xmin": xmin_m,
    "xmax": xmax_m,
    "ymin": ymin_m,
    "ymax": ymax_m,
    "width": width_m,
    "height": height_m,
    "area": area_m2
})

df_converted


FileNotFoundError: No se encontró el archivo CSV en la ruta: ../data/database/test/tif_1x_res/

In [17]:
# Definir nuevo dataframe por etiquetas
df_lemon = df_converted[df_converted["label"] == "Lemon"]
df_orange = df_converted[df_converted["label"] == "Orange"]
df_tree = df_converted[df_converted["label"] == "Tree"]

In [19]:
df_lemon.describe()

,xmin,xmax,ymin,ymax,width,height,area
count,740.000000,740.000000,7.400000e+02,740.000000,740.000000,740.000000,740.000000
mean,46.253519,49.063497,4.417799e+01,46.923422,2.809978,2.745428,8.583370
std,30.509013,30.379391,2.918483e+01,29.125926,1.011096,1.016791,5.799360
min,0.000000,0.319700,-1.268904e-16,0.575460,0.169874,0.127406,0.021643
25%,18.943510,22.304041,1.848976e+01,21.419899,2.015339,1.916883,3.881552
50%,44.669204,47.328207,4.253788e+01,45.152932,2.741962,2.621138,7.093360
75%,69.220375,72.341344,6.704587e+01,69.518914,3.590055,3.543538,12.432874
max,117.005958,118.448850,1.128131e+02,114.292750,6.125009,5.758538,29.456074


In [20]:
df_orange.describe()

,xmin,xmax,ymin,ymax,width,height,area
count,1.251000e+03,1251.000000,1.251000e+03,1251.000000,1251.000000,1251.000000,1251.000000
mean,4.035254e+01,44.540185,4.681428e+01,51.204639,4.187646,4.390357,19.449581
std,2.836272e+01,28.502098,2.874162e+01,28.681164,1.194055,1.248026,9.115817
min,-1.526146e-16,0.455887,-1.296412e-16,0.610668,0.127880,0.029993,0.005398
25%,1.614868e+01,20.890532,2.207364e+01,26.605041,3.630291,3.823050,13.533569
50%,3.668286e+01,41.045139,4.709187e+01,51.621282,4.272816,4.582492,19.595456
75%,6.219479e+01,66.933962,6.869258e+01,73.477969,4.932157,5.224887,25.034517
max,1.122982e+02,113.845170,1.074153e+02,108.186480,8.993977,8.020612,62.771899


In [21]:
df_tree.describe()

,xmin,xmax,ymin,ymax,width,height,area
count,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000
mean,48.605415,52.866773,53.154432,57.427325,4.261358,4.272893,29.985468
std,33.868191,33.466445,33.850174,33.350606,3.649990,3.459036,63.899294
min,0.000000,0.330570,0.000000,1.788480,0.330570,0.841638,0.437107
25%,22.119255,26.262252,22.414416,26.125683,2.060797,2.240651,4.880573
50%,41.907714,48.554981,57.201417,60.028546,3.276900,3.389302,11.934014
75%,73.759874,77.469380,83.875228,89.617161,4.605068,4.893139,22.724819
max,112.258378,118.448850,109.308636,113.602933,21.345518,20.693615,441.715916
